In [ ]:
# See: https://python.langchain.com/docs/use_cases/question_answering/quickstart
!pip install python-dotenv
!pip install --upgrade langchain langchain-community langchainhub langchain-openai langchain-chroma bs4 huggingface_hub
!pip install transformers
!pip install einops
!pip install unstructured
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import userdata
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = serdata.get('LANGCHAIN_API_KEY')

In [ ]:

from dotenv import load_dotenv
load_dotenv()

False

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import HuggingFaceHub
from langchain_community.llms import HuggingFaceEndpoint

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    endpoint_url="https://csmbxk03mo0rh79k.us-east-1.aws.endpoints.huggingface.cloud", # llama 3 8b instruct
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Load document:
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/Large_language_model",),
)
docs = loader.load()

In [ ]:
from langchain_community.embeddings import HuggingFaceHubEmbeddings
embedding = HuggingFaceHubEmbeddings(
    model="https://s4trphd3wihrma8o.us-east-1.aws.endpoints.huggingface.cloud", # microsoft's e5 large instruct
)

In [ ]:
# chunk and index the contents:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
Chroma().delete_collection()
for v in splits:
  vectorstore = Chroma.from_documents(documents=[v], embedding=embedding)

# Retrieve and generate
retriever = vectorstore.as_retriever()

# rlm/rag-prompt is a prompt template

# You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

# Question: {question}

# Context: {context}

# Answer:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()} # the retriever takes the human question input, search in the vecotr db for relevant content
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What could LLM be used for?")

' A large language model (LLM) can be used for various tasks, such as text generation, classification, and prompt-engineering, as well as serving as a world model, rollout heuristic, or planner for complex action sequences. It can also be used to score observations for their interestingness or propose tasks for curriculum learning. Additionally, LLMs have been used to demonstrate abilities such as mathematical reasoning and solving novel tasks that span multiple domains.'